<a href="https://colab.research.google.com/github/S-Gunjan/BA820-A1-OkCupid/blob/main/1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## BA820 Project: OkCupid Profiles

In [1]:
# Importing libraries and mounting the drive

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
import seaborn as sns
import statsmodels.formula.api as smf
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
!pip install -U googlemaps
drive.mount('/content/drive')
data_folder = '/content/drive/MyDrive/Colab Notebooks/BA820/Data/'

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40711 sha256=bf899384ec9cdaad679ccc2488fc2f367dc02712e3da9009f928baa35194f0c4
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps
Mounted at /content/drive


In [2]:
data = pd.read_csv(data_folder+'okcupid_profiles.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59946 entries, 0 to 59945
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          59946 non-null  int64  
 1   status       59946 non-null  object 
 2   sex          59946 non-null  object 
 3   orientation  59946 non-null  object 
 4   body_type    54650 non-null  object 
 5   diet         35551 non-null  object 
 6   drinks       56961 non-null  object 
 7   drugs        45866 non-null  object 
 8   education    53318 non-null  object 
 9   ethnicity    54266 non-null  object 
 10  height       59943 non-null  float64
 11  income       59946 non-null  int64  
 12  job          51748 non-null  object 
 13  last_online  59946 non-null  object 
 14  location     59946 non-null  object 
 15  offspring    24385 non-null  object 
 16  pets         40025 non-null  object 
 17  religion     39720 non-null  object 
 18  sign         48890 non-null  object 
 19  smok

In [4]:
data.describe()

,age,height,income
count,59946.000000,59943.000000,59946.000000
mean,32.340290,68.295281,20033.222534
std,9.452779,3.994803,97346.192104
min,18.000000,1.000000,-1.000000
25%,26.000000,66.000000,-1.000000
50%,30.000000,68.000000,-1.000000
75%,37.000000,71.000000,-1.000000
max,110.000000,95.000000,1000000.000000


In [5]:
num_rows, num_columns = data.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

Number of rows: 59946
Number of columns: 31


In [6]:
data.corr()

<ipython-input-6-c44ded798807>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.corr()


,age,height,income
age,1.000000,-0.022262,-0.001004
height,-0.022262,1.000000,0.065049
income,-0.001004,0.065049,1.000000


In [7]:
data.isna().sum()

age                0
status             0
sex                0
orientation        0
body_type       5296
diet           24395
drinks          2985
drugs          14080
education       6628
ethnicity       5680
height             3
income             0
job             8198
last_online        0
location           0
offspring      35561
pets           19921
religion       20226
sign           11056
smokes          5512
speaks            50
essay0          5488
essay1          7572
essay2          9638
essay3         11476
essay4         10537
essay5         10850
essay6         13771
essay7         12451
essay8         19225
essay9         12603
dtype: int64

In [8]:
data["income"]=data["income"].replace(-1, 0)

Convert non-numeric values to NaN, Removes rows with NaN values, and caps ages above 100 to 100 in the "Age" column

In [9]:
data['age'] = pd.to_numeric(data['age'], errors='coerce')

data = data.dropna(subset=['age'])

data['age'] = data['age'].apply(lambda x: x if x <= 100 else 100)

data["age"]

0        22
1        35
2        38
3        23
4        29
         ..
59941    59
59942    24
59943    42
59944    27
59945    39
Name: age, Length: 59946, dtype: int64

Map various body type categories to standardized categories by replacing the original categories with the standardized ones then fill any remaining missing values in the "body_type" column with 'rather not say' and count the occurrences of each category in the "body_type" column to display the distribution of body types.





In [10]:
category_mapping = {
    'curvy': 'curvy',
    'Thin': 'thin', 'skinny': 'thin',
    'athletic': 'fit', 'fit': 'fit',
    'rather not say': 'rather not say', 'nan': 'rather not say',
    'Overweight': 'Overweight',
    'used up': 'average', 'average': 'average',
    'Jacked': 'jacked',
    'A little extra': 'A little extra',
    'full figured': 'full figured'
}

data['body_type'] = data['body_type'].map(category_mapping)

data['body_type'] = data['body_type'].fillna('rather not say')

data['body_type'].value_counts()

fit               24530
average           15007
rather not say    13699
curvy              3924
thin               1777
full figured       1009
Name: body_type, dtype: int64

Map various diet categories to standardized categories by replacing the original categories with the standardized ones then fill any remaining missing values in the "diet" column with 'Prefer not to say' and count the occurrences of each category in the "diet" column to display the distribution of diets.

In [11]:
diet_mapping = {
    'strictly vegetarian': 'vegetarian',
    'vegetarian': 'vegetarian',
    'mostly vegetarian': 'vegetarian',
    'strictly anything': 'Prefer not to say',
    'anything': 'Prefer not to say',
    'mostly anything': 'Prefer not to say',
    'mostly other': 'Prefer not to say',
    'strictly other': 'Prefer not to say',
    'other': 'Prefer not to say',
    'strictly vegan': 'vegan',
    'mostly vegan': 'vegan',
    'vegan': 'vegan',
    'strictly halal': 'halal',
    'halal': 'halal',
    'mostly halal': 'halal',
    'strictly kosher': 'kosher',
    'kosher': 'kosher',
    'mostly kosher': 'kosher'
}

data['diet'] = data['diet'].map(diet_mapping)

data['diet'].fillna('Prefer not to say', inplace=True)

data['diet'].value_counts()

Prefer not to say    54066
vegetarian            4986
vegan                  702
kosher                 115
halal                   77
Name: diet, dtype: int64

Fill missing values with 'prefer not to say'. Then, it counts the occurrences of each category in the "drinks" column to display the distribution of drink preferences.

In [12]:
data['drinks'].fillna('prefer not to say', inplace=True)
data['drinks'].value_counts()

socially             41780
rarely                5957
often                 5164
not at all            3267
prefer not to say     2985
very often             471
desperately            322
Name: drinks, dtype: int64

Fill missing values with 'prefer not to say'. Then, it counts the occurrences of each category in the "drugs" column to display the distribution of drug preferences.

In [13]:
data['drugs'].fillna('prefer not to say', inplace=True)
data['drugs'].value_counts()

never                37724
prefer not to say    14080
sometimes             7732
often                  410
Name: drugs, dtype: int64

Map various education categories to standardized categories by replacing the original categories with the standardized ones then fill any remaining missing values in the "education" column with 'Prefer not to say' and count the occurrences of each category in the "education" column to display the distribution of education.

In [14]:
education_mapping = {
    'working on high school': 'In high school',
    'high school': 'In high school',
    'working on college/university': 'In college',
    'working on two-year college': 'In college',
    'college/university': 'In college',
    'two-year college': 'In college',
    'working on med school': 'In college',
    'med school': 'In college',
    'law school': 'In college',
    'working on law school': 'In college',
    'working on masters program': 'In grad school',
    'masters program': 'In grad school',
    'graduated from high school': 'High school degree',
    'graduated from college/university': 'College degree',
    'graduated from two-year college': 'College degree',
    'graduated from med school': 'College degree',
    'graduated from law school': 'College degree',
    'graduated from masters program': 'Graduate degree',
    'ph.d program': 'PHD',
    'working on ph.d program': 'PHD',
    'graduated from ph.d program': 'PHD title',
    'dropped out of ph.d program': 'PHD dropout',
    'dropped out of med school': 'Dropped out of high school',
    'dropped out of high school': 'Dropped out of high school',
    'dropped out of college/university': 'Dropped out of college',
    'dropped out of two-year college': 'Dropped out of college',
    'dropped out of law school': 'Dropped out of college',
    'dropped out of masters program': 'Dropped out of grad school',
    'working on space camp': 'In space camp',
    'space camp': 'In space camp',
    'graduated from space camp': 'Graduated from space camp',
    'dropped out of space camp': 'Dropped out of space camp'
}

data['education'] = data['education'].map(education_mapping)

data['education'].fillna('Prefer not to say', inplace=True)

data['education'].value_counts()

College degree                27058
Graduate degree                8961
In college                     8320
Prefer not to say              6628
In grad school                 1819
High school degree             1428
PHD title                      1272
Dropped out of college         1204
PHD                            1009
Graduated from space camp       657
Dropped out of space camp       523
In space camp                   503
In high school                  183
Dropped out of grad school      140
PHD dropout                     127
Dropped out of high school      114
Name: education, dtype: int64

Fill missing values with 'prefer not to say'. Then, it counts the occurrences of each category in the "ethnicity" column to display the distribution of ethnicity preferences.

In [15]:
data['ethnicity'].fillna('prefer not to say', inplace=True)
data['ethnicity'].value_counts()

white                                                                 32831
asian                                                                  6134
prefer not to say                                                      5680
hispanic / latin                                                       2823
black                                                                  2008
                                                                      ...  
middle eastern, indian, white                                             1
asian, middle eastern, black, white, other                                1
asian, middle eastern, indian, hispanic / latin, white, other             1
black, native american, indian, pacific islander, hispanic / latin        1
asian, black, indian                                                      1
Name: ethnicity, Length: 218, dtype: int64

 Handle missing values in the "height" column by filling them with the median height value.

In [16]:
median_height = data['height'].median()

data['height'].fillna(median_height, inplace=True)

remaining_nan = data['height'].isna().sum()
median_height

68.0

Fill missing values with 'rather not say'in the "job" column.

In [17]:
data['job'] = data['job'].fillna('rather not say')

In [18]:
(data['last_online'].head())

0    2012-06-28-20-30
1    2012-06-29-21-41
2    2012-06-27-09-10
3    2012-06-28-14-22
4    2012-06-27-21-26
Name: last_online, dtype: object

Convert the values in the "last_online" column to datetime objects using the to_datetime

In [19]:
data['last_online'] = pd.to_datetime(data['last_online'], format='%Y-%m-%d-%H-%M')

Map various offspring categories to standardized categories by replacing the original categories with the standardized ones then fill any remaining missing values in the "offspring" column with 'Prefer not to say' and count the occurrences of each category in the "education" column to display the distribution of offsprings.

In [20]:
offspring_mapping = {
    "doesn't have kids, but might want them": "Doesnt have kids but might want them",
    "doesn't have kids, but wants them": "Doesnt have kids but wants them",
    "doesn't have kids, and doesn't want any": "Doesnt have kids and doesnt want them",
    "has a kid, but doesn't want more": "Has kids and doesnt want more",
    "has kids, but doesn't want more": "Has kids and doesnt want more",
    'has a kid, and might want more': 'Has kids and might want more',
    'has kids, and might want more': 'Has kids and might want more',
    'has kids, and wants more': 'Has kids and wants more',
    'has a kid, and wants more': 'Has kids and wants more',
    'has a kid': 'Has kids',
    'has kids': 'Has kids',
}

data['offspring'] = data['offspring'].map(offspring_mapping)

data['offspring'].fillna('Prefer not to say', inplace=True)

data['offspring'].value_counts()

Prefer not to say                        46455
Doesnt have kids but might want them      3875
Has kids                                  3764
Doesnt have kids but wants them           3565
Doesnt have kids and doesnt want them     1132
Has kids and doesnt want more              717
Has kids and might want more               346
Has kids and wants more                     92
Name: offspring, dtype: int64

Fill missing values with 'prefer not to say' in the "pets" column.

In [21]:
data['pets'].fillna('prefer not to say', inplace=True)
data['pets'].value_counts()

prefer not to say                  19921
likes dogs and likes cats          14814
likes dogs                          7224
likes dogs and has cats             4313
has dogs                            4134
has dogs and likes cats             2333
likes dogs and dislikes cats        2029
has dogs and has cats               1474
has cats                            1406
likes cats                          1063
has dogs and dislikes cats           552
dislikes dogs and likes cats         240
dislikes dogs and dislikes cats      196
dislikes cats                        122
dislikes dogs and has cats            81
dislikes dogs                         44
Name: pets, dtype: int64

Fill missing values with 'prefer not to say' in the "religion" column.

In [22]:
data['religion'].fillna('prefer not to say', inplace=True)
data['religion'].value_counts()

prefer not to say                             20226
agnosticism                                    2724
other                                          2691
agnosticism but not too serious about it       2636
agnosticism and laughing about it              2496
catholicism but not too serious about it       2318
atheism                                        2175
other and laughing about it                    2119
atheism and laughing about it                  2074
christianity                                   1957
christianity but not too serious about it      1952
other but not too serious about it             1554
judaism but not too serious about it           1517
atheism but not too serious about it           1318
catholicism                                    1064
christianity and somewhat serious about it      927
atheism and somewhat serious about it           848
other and somewhat serious about it             846
catholicism and laughing about it               726
judaism and 

Fill missing values with 'prefer not to say' in the "sign" column.

In [23]:
data['sign'].fillna('prefer not to say', inplace=True)
data['sign'] = data['sign'].apply(lambda x: x.replace('&rsquo;', ''))
data['sign'].value_counts()

prefer not to say                         11056
gemini and its fun to think about          1782
scorpio and its fun to think about         1772
leo and its fun to think about             1692
libra and its fun to think about           1649
taurus and its fun to think about          1640
cancer and its fun to think about          1597
pisces and its fun to think about          1592
sagittarius and its fun to think about     1583
virgo and its fun to think about           1574
aries and its fun to think about           1573
aquarius and its fun to think about        1503
virgo but it doesnt matter                 1497
leo but it doesnt matter                   1457
cancer but it doesnt matter                1454
gemini but it doesnt matter                1453
taurus but it doesnt matter                1450
aquarius but it doesnt matter              1408
libra but it doesnt matter                 1408
capricorn and its fun to think about       1376
sagittarius but it doesnt matter        

Fill missing values with 'prefer not to say'in the "smokes" column.

In [24]:
data['smokes'].fillna('prefer not to say', inplace=True)
data['smokes'].value_counts()

no                   43896
prefer not to say     5512
sometimes             3787
when drinking         3040
yes                   2231
trying to quit        1480
Name: smokes, dtype: int64

Fill missing values with 'prefer not to say'in the "speaks" column.

In [25]:
data['speaks'].fillna('prefer not to say', inplace=True)
data['speaks'].value_counts()

english                                                                21828
english (fluently)                                                      6628
english (fluently), spanish (poorly)                                    2059
english (fluently), spanish (okay)                                      1917
english (fluently), spanish (fluently)                                  1288
                                                                       ...  
english (fluently), urdu (poorly), japanese (poorly), french (okay)        1
english, spanish, hindi, c++                                               1
english (fluently), japanese (okay), thai (okay), chinese (poorly)         1
english (fluently), french (okay), italian (okay), hebrew (okay)           1
english (fluently), french, farsi                                          1
Name: speaks, Length: 7648, dtype: int64

Combine multiple essay columns into a single column named 'combined_essays', and fill any missing values with a standard placeholder text ('prefer not to say').

In [26]:
essay_columns = [
    'essay0',
    'essay1',
    'essay2',
    'essay3',
    'essay4',
    'essay5',
    'essay6',
    'essay7',
    'essay8',
    'essay9'
]

for column in essay_columns:
    data[column] = data[column].fillna('prefer not to say')

data['combined_essays'] = data[essay_columns].apply(' '.join, axis=1)

data[['combined_essays']].head()

,combined_essays
0,about me: i would love to think that i was so...
1,i am a chef: this is what that means. 1. i am ...
2,"i'm not ashamed of much, but writing public te..."
3,i work in a library and go to school. . . read...
4,hey how's it going? currently vague on the pro...


Rename the specified columns inplace

In [27]:
column_rename_mapping = {
    'essay0': 'self_summary',
    'essay1': 'current_activities',
    'essay2': 'skills_talents',
    'essay3': 'first_noticeable',
    'essay4': 'favorites',
    'essay5': 'essentials',
    'essay6': 'thoughts',
    'essay7': 'friday_night',
    'essay8': 'private_admission',
    'essay9': 'message_reasons'
}

data.rename(columns=column_rename_mapping, inplace=True)

data.head()

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,current_activities,skills_talents,first_noticeable,favorites,essentials,thoughts,friday_night,private_admission,message_reasons,combined_essays
0,22,single,m,straight,rather not say,Prefer not to say,socially,never,In college,"asian, white",...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...","books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...,about me: i would love to think that i was so...
1,35,single,m,straight,average,Prefer not to say,often,sometimes,In space camp,white,...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,prefer not to say,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories. my b...,prefer not to say,prefer not to say,i am very open and will share just about anyth...,prefer not to say,i am a chef: this is what that means. 1. i am ...
2,38,available,m,straight,rather not say,Prefer not to say,socially,prefer not to say,Graduate degree,prefer not to say,...,"i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,okay this is where the cultural matrix gets so...,movement conversation creation contemplation t...,prefer not to say,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ...","i'm not ashamed of much, but writing public te..."
3,23,single,m,straight,rather not say,vegetarian,socially,prefer not to say,In college,white,...,reading things written by old dead people,playing synthesizers and organizing books acco...,socially awkward but i do my best,"bataille, celine, beckett. . . lynch, jarmusch...",prefer not to say,cats and german philosophy,prefer not to say,prefer not to say,you feel so inclined.,i work in a library and go to school. . . read...
4,29,single,m,straight,fit,Prefer not to say,socially,never,College degree,"asian, black, other",...,work work work work + play,creating imagery to look at: http://bagsbrown....,i smile a lot and my inquisitive nature,"music: bands, rappers, musicians at the moment...",prefer not to say,prefer not to say,prefer not to say,prefer not to say,prefer not to say,hey how's it going? currently vague on the pro...
